In [46]:
import numpy as np
import pprint
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv

In [47]:
pp = pprint.PrettyPrinter(indent=2)
env = GridworldEnv()

In [48]:
def policy_eval(policy, env, discount_factor=1.0, theta=0.00001):
    """
    Evaluate a policy given an environment and a full description of the environment's dynamics.
    
    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI env,
            env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is the number of states in the environment. 
            env.nA is the number of actions in the environment.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: Gamma discount factor.
    
    Returns:
        Vector of length env.nS representing the value function.
    """
    # Start with a random (all 0) value function
    V_old = np.zeros(env.nS)
    while True:
        #print("iteration: _____________________________________________________")
        khalas = 0
        V_new = np.zeros(env.nS)
        for i in range(env.nS):
            if i == 0 or i == env.nS - 1:
                continue
            for a in range(env.nA):
                V_new[i] += policy[i][a] * (env.P[i][a][2] + V_old[env.P[i][a][1]])
                #print("state:", i, ", action:", a)
                #print("V_old:", V_old[i], ", V_new:", V_new[i])
            
        for check in range(env.nS):
            if abs(V_old[check] - V_new[check]) > theta:
                break
        else:
            break
        for cnt in range(env.nS):
            V_old[cnt] = V_new[cnt]
        
    return V_new

In [43]:
# Taken from Policy Evaluation Exercise!

def policy_eval(policy, env, discount_factor=1.0, theta=0.00001):
    """
    Evaluate a policy given an environment and a full description of the environment's dynamics.
    
    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment. 
            env.nA is a number of actions in the environment.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: Gamma discount factor.
    
    Returns:
        Vector of length env.nS representing the value function.
    """
    # Start with a random (all 0) value function
    V = np.zeros(env.nS)
    while True:
        delta = 0
        # For each state, perform a "full backup"
        for s in range(env.nS):
            v = 0
            # Look at the possible next actions
            for a, action_prob in enumerate(policy[s]):
                # For each action, look at the possible next states...
                for  prob, next_state, reward, done in env.P[s][a]:
                    # Calculate the expected value
                    v += action_prob * prob * (reward + discount_factor * V[next_state])
            # How much our value function changed (across any states)
            delta = max(delta, np.abs(v - V[s]))
            V[s] = v
        # Stop evaluating once our value function change is below a threshold
        if delta < theta:
            break
    return V

In [51]:
def policy_iteration(env, policy_eval_fn=policy_eval, discount_factor=1.0):
    """
    Policy iteration Algorithm. Iteratively evaluates and improves a policy
    until an optimal policy is found.
    
    Args:
        env: The OpenAI envrionment.
        
        policy_eval_fn: Policy Evaluation function that takes 3 arguments:
            policy, env, discount_factor.
            
        discount_factor: gamma discount factor.
        
    Returns:
        A tuple (optimal_policy, optimal_V). 
        
        optimal_policy is the optimal policy, a matrix of shape [S, A] where each state s
        contains a valid probability distribution over actions.
        
        optimal_V is the value function for the optimal policy.
        
    """
    # Start with a random policy
    policy = np.ones([env.nS, env.nA]) / env.nA
    
    while True:
        v_old = policy_eval_fn(policy, env)
        policy = np.zeros([env.nS, env.nA])
        for s in range(env.nS):
            possible_returns_of_a_state = []
            for a in range(env.nA):
                possible_returns_of_a_state.append(env.P[s][a][2] + v_old[env.P[s][a][1]])
            max_return = max(possible_returns_of_a_state)
            new_transitions = np.zeros(len(possible_returns_of_a_state))
            for i in range(len(possible_returns_of_a_state)):
                if possible_returns_of_a_state[i] == max_return:
                    new_transitions[i] = 1
            policy[s] = new_transitions / np.sum(new_transitions)
        v = policy_eval_fn(policy, env)
        for check in range(env.nS):
            if abs(v_old[check] - v[check]) > 0.00001:
                break
        else:
            break
    
    return policy, v

In [52]:
policy, v = policy_iteration(env)
print("Policy Probability Distribution:")
print(policy)
print("")

print("Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):")
print(np.reshape(np.argmax(policy, axis=1), env.shape))
print("")

print("Value Function:")
print(v)
print("")

print("Reshaped Grid Value Function:")
print(v.reshape(env.shape))
print("")



Policy Probability Distribution:
[[0.25 0.25 0.25 0.25]
 [0.   0.   0.   1.  ]
 [0.   0.   0.   1.  ]
 [0.   0.   0.5  0.5 ]
 [1.   0.   0.   0.  ]
 [0.5  0.   0.   0.5 ]
 [0.25 0.25 0.25 0.25]
 [0.   0.   1.   0.  ]
 [1.   0.   0.   0.  ]
 [0.25 0.25 0.25 0.25]
 [0.   0.5  0.5  0.  ]
 [0.   0.   1.   0.  ]
 [0.5  0.5  0.   0.  ]
 [0.   1.   0.   0.  ]
 [0.   1.   0.   0.  ]
 [0.25 0.25 0.25 0.25]]

Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):
[[0 3 3 2]
 [0 0 0 2]
 [0 0 1 2]
 [0 1 1 0]]

Value Function:
[ 0. -1. -2. -3. -1. -2. -3. -2. -2. -3. -2. -1. -3. -2. -1.  0.]

Reshaped Grid Value Function:
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]



In [53]:
# Test the value function
expected_v = np.array([ 0, -1, -2, -3, -1, -2, -3, -2, -2, -3, -2, -1, -3, -2, -1,  0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)